<a href="https://colab.research.google.com/github/MakkuranoNeko/Portfolio/blob/main/%D0%AF%D0%BD%D0%B4%D0%B5%D0%BA%D1%81_%D0%BD%D0%B5%D0%B4%D0%B2%D0%B8%D0%B6%D0%B8%D0%BC%D0%BE%D1%81%D1%82%D1%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


 **Описание проекта** 

В вашем распоряжении данные сервиса Яндекс.Недвижимость — архив объявлений о продаже квартир в Санкт-Петербурге и соседних населённых пунктах за несколько лет. Нужно научиться определять рыночную стоимость объектов недвижимости.

Задача — установить параметры. Это позволит построить автоматизированную систему: она отследит аномалии и мошенническую деятельность.

По каждой квартире на продажу доступны два вида данных. Первые вписаны пользователем, вторые — получены автоматически на основе картографических данных. Например, расстояние до центра, аэропорта, ближайшего парка и водоёма.


# **0. Импортируем данные и нужные библиотеки**

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import statistics as st
import matplotlib.pyplot as plt
import plotly.express as px
import math
import datetime
import calendar
import scipy.stats as stats
import pylab as pl

In [ ]:
data = pd.read_csv('real_estate_data.csv', sep = '\t')

# **1. Общее описание данных**

Для начала посмотрим на первые пять значений датасета.

In [ ]:
data.head()

,total_images,last_price,total_area,first_day_exposition,rooms,ceiling_height,floors_total,living_area,floor,is_apartment,studio,open_plan,kitchen_area,balcony,locality_name,airports_nearest,cityCenters_nearest,parks_around3000,parks_nearest,ponds_around3000,ponds_nearest,days_exposition
0,20,13000000.0,108.0,2019-03-07T00:00:00,3,2.70,16.0,51.0,8,NaN,False,False,25.0,NaN,Санкт-Петербург,18863.0,16028.0,1.0,482.0,2.0,755.0,NaN
1,7,3350000.0,40.4,2018-12-04T00:00:00,1,NaN,11.0,18.6,1,NaN,False,False,11.0,2.0,посёлок Шушары,12817.0,18603.0,0.0,NaN,0.0,NaN,81.0
2,10,5196000.0,56.0,2015-08-20T00:00:00,2,NaN,5.0,34.3,4,NaN,False,False,8.3,0.0,Санкт-Петербург,21741.0,13933.0,1.0,90.0,2.0,574.0,558.0
3,0,64900000.0,159.0,2015-07-24T00:00:00,3,NaN,14.0,NaN,9,NaN,False,False,NaN,0.0,Санкт-Петербург,28098.0,6800.0,2.0,84.0,3.0,234.0,424.0
4,2,10000000.0,100.0,2018-06-19T00:00:00,2,3.03,14.0,32.0,13,NaN,False,False,41.0,NaN,Санкт-Петербург,31856.0,8098.0,2.0,112.0,1.0,48.0,121.0


Выясним общую информацию, о количестве данных и об их типе.

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23699 entries, 0 to 23698
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   total_images          23699 non-null  int64  
 1   last_price            23699 non-null  float64
 2   total_area            23699 non-null  float64
 3   first_day_exposition  23699 non-null  object 
 4   rooms                 23699 non-null  int64  
 5   ceiling_height        14504 non-null  float64
 6   floors_total          23613 non-null  float64
 7   living_area           21796 non-null  float64
 8   floor                 23699 non-null  int64  
 9   is_apartment          2775 non-null   object 
 10  studio                23699 non-null  bool   
 11  open_plan             23699 non-null  bool   
 12  kitchen_area          21421 non-null  float64
 13  balcony               12180 non-null  float64
 14  locality_name         23650 non-null  object 
 15  airports_nearest   

Отдельно выведем количество пустых ячеек в наших данных.

In [ ]:
for i in data.columns:
    print(i, data[i].isnull().sum(), round(data[i].isnull().sum() / len(data) * 100, 2), '%' )

total_images 0 0.0 %
last_price 0 0.0 %
total_area 0 0.0 %
first_day_exposition 0 0.0 %
rooms 0 0.0 %
ceiling_height 9195 38.8 %
floors_total 86 0.36 %
living_area 1903 8.03 %
floor 0 0.0 %
is_apartment 20924 88.29 %
studio 0 0.0 %
open_plan 0 0.0 %
kitchen_area 2278 9.61 %
balcony 11519 48.61 %
locality_name 49 0.21 %
airports_nearest 5542 23.38 %
cityCenters_nearest 5519 23.29 %
parks_around3000 5518 23.28 %
parks_nearest 15620 65.91 %
ponds_around3000 5518 23.28 %
ponds_nearest 14589 61.56 %
days_exposition 3181 13.42 %


Основная часть пропущенных данных приходится на столбцы: is_apartment (88% пропущенных данных), balcony (49%), ceiling_height (39%), parks_nearest (66%), ponds_nearest (62%). 

Также стоит отметить, что формат столбца first_day_exposition должен быть датой.

# **2. Пустые значения в данном dataFrame.**

**План обработки пустых значений**
1. Если количество балконов не указано, то скорее всего их нет. Иначе бы владелец обязательно указал этот фактор, чтоб повысить привлекательность квартиры для продажи. Заменим значения на 0.
2. Так как мы чаще всего имеем стандартную планировку квартир, то может заменить пропущенные значения высоты потолка на среднее значение.
3. Колонка апартаментов имеет слишком много пропущенных значений и не несет из-за этого полезную информацию. Удалим из рассмотрения.
4. Количество ближайших парков и прудов имеет много пропусков и тоже удалим.
5. Заменим пропуски в количестве ближайших парков и прудов на 0.

2.1 Замена "NaN" для количества балконов

In [ ]:
data['balcony'] = data.balcony.fillna(0)

In [ ]:
data['balcony'].isnull().sum()

0

2.2 Замена пропущенных данных о высоте потолков

In [ ]:
data['ceiling_height'].mean()

2.7714988968559835

In [ ]:
data['ceiling_height'] = data.ceiling_height.fillna(2.7)

In [ ]:
data['ceiling_height'].isnull().sum()

0


2.3 Очистка от ненужных столбцов

In [ ]:
new_columns = list(data.columns)


In [ ]:
print(new_columns)

['total_images', 'last_price', 'total_area', 'first_day_exposition', 'rooms', 'ceiling_height', 'floors_total', 'living_area', 'floor', 'is_apartment', 'studio', 'open_plan', 'kitchen_area', 'balcony', 'locality_name', 'airports_nearest', 'cityCenters_nearest', 'parks_around3000', 'parks_nearest', 'ponds_around3000', 'ponds_nearest', 'days_exposition']


In [ ]:
new_columns.remove('is_apartment')
new_columns.remove('parks_nearest')
new_columns.remove('ponds_nearest')

In [ ]:
data = data[new_columns]

2.4 Замена "NaN" для количества парков и прудов.

In [ ]:
data['parks_around3000'] = data.parks_around3000.fillna(0)
data['ponds_around3000'] = data.ponds_around3000.fillna(0)

# **3. Смена формата данных.**

После заполнения пропусков данных видно, что столбец "balcony", 'parks_around3000' и 'ponds_around3000' все еще имеет формат float вместо int и занимает больше места, чем мог бы. Также надо поменять столбец "first_day_exposition" на datetime и очистить от времени, так как для всех дней время появления объявления указано как T00:00:00.

Отдельно отметим, что остальные столбцы имеет пропуски и это не позволяет сменить их формат с float на int.

In [ ]:
data['balcony'] = data['balcony'].astype('int64')
data['parks_around3000'] = data['parks_around3000'].astype('int64')
data['ponds_around3000'] = data['ponds_around3000'].astype('int64')

Исправим столбец с датой на формат даты и очистим от времени.

In [ ]:
data['first_day_exposition'] = [string[:10] for string in data['first_day_exposition']]
data['first_day_exposition'] = pd.to_datetime(data['first_day_exposition'])

Теперь тип данных определен верно.

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23699 entries, 0 to 23698
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   total_images          23699 non-null  int64         
 1   last_price            23699 non-null  float64       
 2   total_area            23699 non-null  float64       
 3   first_day_exposition  23699 non-null  datetime64[ns]
 4   rooms                 23699 non-null  int64         
 5   ceiling_height        23699 non-null  float64       
 6   floors_total          23613 non-null  float64       
 7   living_area           21796 non-null  float64       
 8   floor                 23699 non-null  int64         
 9   studio                23699 non-null  bool          
 10  open_plan             23699 non-null  bool          
 11  kitchen_area          21421 non-null  float64       
 12  balcony               23699 non-null  int64         
 13  locality_name   

# **4. Очистка данных от выбросов**

Чтобы определить есть ли выбросы и для каких параметров они характерны, построим описательную статистику.

In [ ]:
data.describe()

,total_images,last_price,total_area,rooms,ceiling_height,floors_total,living_area,floor,kitchen_area,balcony,airports_nearest,cityCenters_nearest,parks_around3000,ponds_around3000,days_exposition
count,23699.000000,2.369900e+04,23699.000000,23699.000000,23699.000000,23613.000000,21796.000000,23699.000000,21421.000000,23699.000000,18157.000000,18180.000000,23699.000000,23699.000000,20518.000000
mean,9.858475,6.541549e+06,60.348651,2.070636,2.743758,10.673824,34.457852,5.892358,10.569807,0.591080,28793.672193,14191.277833,0.469049,0.590911,180.888634
std,5.682529,1.088701e+07,35.654083,1.078405,0.987138,6.597173,22.030445,4.885249,5.905438,0.959298,12630.880622,8608.386210,0.748534,0.883999,219.727988
min,0.000000,1.219000e+04,12.000000,0.000000,1.000000,1.000000,2.000000,1.000000,1.300000,0.000000,0.000000,181.000000,0.000000,0.000000,1.000000
25%,6.000000,3.400000e+06,40.000000,1.000000,2.600000,5.000000,18.600000,2.000000,7.000000,0.000000,18585.000000,9238.000000,0.000000,0.000000,45.000000
50%,9.000000,4.650000e+06,52.000000,2.000000,2.700000,9.000000,30.000000,4.000000,9.100000,0.000000,26726.000000,13098.500000,0.000000,0.000000,95.000000
75%,14.000000,6.800000e+06,69.900000,3.000000,2.700000,16.000000,42.300000,8.000000,12.000000,1.000000,37273.000000,16293.000000,1.000000,1.000000,232.000000
max,50.000000,7.630000e+08,900.000000,19.000000,100.000000,60.000000,409.700000,33.000000,112.000000,5.000000,84869.000000,65968.000000,3.000000,3.000000,1580.000000


**Общие выводы по столбцам:**

Выделим основные столбцы для работы: total_images (число фотографий объекта), last_price (цена продажи), total_area (общая площадь жилья), rooms (число комнат), ceiling_height (высота потолков), floors_total (число этажей), living_area (площадь жилой части), floor (этаж квартиры), kitchen_area (площадь кухни)    


Остальные столбцы не несут качественно полезную информацию и мы не будем по ним ограничивать.

Для каждого столбца мы построим box plot, чтоб визуально оценить выбросы. И после этого оценим какой процент данных находится вне 99% с помощью квантиля.


4.1 Обработка данных столбца last_price

In [ ]:
fig = px.box(data, y="last_price")
fig.show()

In [ ]:
data["last_price"].quantile(0.99)

36000000.0

4.2 Обработка данных столбца total_area

In [ ]:
fig = px.box(data, y="total_area")
fig.show()

In [ ]:
data["total_area"].quantile(0.998)

321.8119999999981

4.3 Обработка данных столбца rooms.

In [ ]:
fig = px.box(data, y="rooms")
fig.show()

4.4 Обработка данных столбца ceiling_height

In [ ]:
fig = px.box(data, y="ceiling_height")
fig.show()

In [ ]:
data.ceiling_height.quantile(0.9986)

5.3

4.5 Обработка данных столбца floors_total

In [ ]:
fig = px.box(data, y="floors_total")
fig.show()

4.6 Обработка данных столбца living_area

In [ ]:
fig = px.box(data, y="living_area")
fig.show()

In [ ]:
data.living_area.quantile(0.9985)

199.30750000000262

**Выводы:**


1.   total_images (число фотографий объекта) - выброс 50 фотографий не портит всю картину в целом и не несет в себе ценной информации.

2.   last_price (цена продажи) - так как мы работаем по большей части с квартирами, то ограничимся стоимостью в 100М. 99.8% всех данных лежат ниже этой стоимости и мы не учтем выброс в 763М и около него. 

3.   total_area (общая площадь жилья) - ограничимся 380м^2. Фактически это первый разрыв в данных и где-то 0.8% всех данных лежат выше 380 м^2.

4.   rooms (число комнат) - ограничимся 6 комнатами по верхней границе boxplot. В данных встречается 0 комнат, укажем такие квартиры как студии. (будет сделано ниже)

5.   ceiling_height (высота потолков) - можно предположить, что квартиры есть двух ярусные с открытым потолком. Значит ограничиваться будем 6м. Это попадает с 99.86% всех  данных. Также удалим квартиры где потолки ниже 2.2 м.

6.   floors_total (число этажей) - В СПб максимальное число этажей 37. На этом и ограничим. Уберем выброс 60 и 52.

7.   living_area (площадь жилой части) - так как мы ограничили выбросы в общей площади, а это созависимый параметр, то оставим все как есть.

8.   Floor (этаж квартиры) - так как мы ограничили все этажи 37. То оставляется все данные.

9.   Kitchen_area (площадь кухни) - аналогично с жилой площадью.


Остальные столбцы не несут качественно полезную информацию и мы не будем по ним ограничивать.


In [ ]:
data = data[(data['last_price'] <= 100000000) & (data['total_area'] <= 380) & 
            (data['rooms'] <= 6) & 
            (data['ceiling_height'] <= 6) & (data['ceiling_height'] >= 2.2) & 
            (data['floors_total'] <= 37)]

Так как мы обрезали данные, то нужно сбросить индексы.

In [ ]:
data = data.reset_index(drop=True)

# **5. Замена пропущенных и некоректных данных**

Посмотрим еще раз на статистику по датафрейму и проверим сохранились ли пропущенные данные для столбца жилой площади и площади кухни.

In [ ]:
data.describe()

,total_images,last_price,total_area,rooms,ceiling_height,floors_total,living_area,floor,kitchen_area,balcony,airports_nearest,cityCenters_nearest,parks_around3000,ponds_around3000,days_exposition
count,23439.000000,2.343900e+04,23439.000000,23439.000000,23439.000000,23439.000000,21586.000000,23439.000000,21234.000000,23439.000000,17930.000000,17951.000000,23439.000000,23439.000000,20310.000000
mean,9.865139,6.151472e+06,59.084318,2.044712,2.713805,10.696233,33.740722,5.884466,10.473049,0.593626,28812.508254,14244.067851,0.464013,0.586202,180.393747
std,5.673260,6.105663e+06,29.838055,1.001474,0.210417,6.585840,19.012289,4.879411,5.628249,0.959507,12652.672420,8598.790160,0.743473,0.880368,219.250538
min,0.000000,1.219000e+04,12.000000,0.000000,2.200000,1.000000,2.000000,1.000000,1.300000,0.000000,0.000000,181.000000,0.000000,0.000000,1.000000
25%,6.000000,3.400000e+06,40.000000,1.000000,2.600000,5.000000,18.600000,2.000000,7.000000,0.000000,18539.000000,9350.000000,0.000000,0.000000,45.000000
50%,9.000000,4.600000e+06,52.000000,2.000000,2.700000,9.000000,30.000000,4.000000,9.100000,0.000000,26777.000000,13138.000000,0.000000,0.000000,95.000000
75%,14.000000,6.700000e+06,69.000000,3.000000,2.700000,16.000000,42.000000,8.000000,12.000000,1.000000,37310.000000,16299.500000,1.000000,1.000000,231.000000
max,50.000000,9.900000e+07,380.000000,6.000000,5.800000,36.000000,255.700000,33.000000,107.000000,5.000000,84869.000000,65968.000000,3.000000,3.000000,1580.000000


**5.1 Замена числа комнат для квартир-студий** 

Для всех студий заменим число комнат на 0, для оставшихся квартир не студий с 0 комнат заменим на 1.

In [ ]:
data['rooms'] = [0 if data.loc[row, 'studio'] and data.loc[row, 'rooms'] != 0 
                 else 1 if not data.loc[row, 'studio'] and data.loc[row, 'rooms'] == 0
                 else data.loc[row, 'rooms'] for row in range(len(data))]

Проверим написанный list comprehention.
Сколько комнат у квартир не студий, и какие уникальные значения имеет столбец числа комнат?



In [ ]:
data[data['studio'] == False].rooms.unique()

array([3, 1, 2, 4, 5, 6])

In [ ]:
data['rooms'].unique()

array([3, 1, 2, 4, 5, 6, 0])

**5.2 Замена пропущенных данных для living_area и kitchen_area**

Так как колонки total_area, living_area и kitchen_area взаимосвяазнны. То мы можем восстановить данные в двух последних с помощью данных из общей площади.

Выделим граничные значения для общей площади:


In [ ]:
print(data['total_area'].max())
data['total_area'].min()

380.0


12.0

Так как в целом квартиры имеют схожие планировки, то перебрав все площади квартир с шагом 10 квадратных метров можем выделить характерные площади для кухни и жилой части.

Для этого мы создадим отдельную колонку 'area_group', описывающую в какой интервал попадает общая площадь жилья.

In [ ]:
for i in range(20, 390, 10):
  temp_data = data[(data['total_area'] >= i - 10) & (data['total_area'] <= i)]
  temp_index = list(temp_data.index)
  data.loc[temp_index, 'area_group'] = str(i - 10) + ' - ' + str(i)

Создадим словарь, состоящий из индекса в виде данных из столбца 'area_group' и связанных с ним данными в виде среднего значения 'living_area' и 'kitchen_area'.

In [ ]:
rest = pd.pivot_table(data, index='area_group', values=['living_area','kitchen_area'], aggfunc='median')

In [ ]:
dict_area = rest.to_dict('index')

Таким образом заменим пропущенные данные из словаря.

In [ ]:
for row in range(len(data)):
  for col in ['living_area', 'kitchen_area']:
    if math.isnan(data.loc[row, col]):
      data.loc[row, col] = dict_area[data.loc[row, 'area_group']][col]

Убедимся, что теперь нет пропущенных значений для этих колонок.

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23439 entries, 0 to 23438
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   total_images          23439 non-null  int64         
 1   last_price            23439 non-null  float64       
 2   total_area            23439 non-null  float64       
 3   first_day_exposition  23439 non-null  datetime64[ns]
 4   rooms                 23439 non-null  int64         
 5   ceiling_height        23439 non-null  float64       
 6   floors_total          23439 non-null  float64       
 7   living_area           23439 non-null  float64       
 8   floor                 23439 non-null  int64         
 9   studio                23439 non-null  bool          
 10  open_plan             23439 non-null  bool          
 11  kitchen_area          23439 non-null  float64       
 12  balcony               23439 non-null  int64         
 13  locality_name   

In [ ]:
data.head(1)

,total_images,last_price,total_area,first_day_exposition,rooms,ceiling_height,floors_total,living_area,floor,studio,open_plan,kitchen_area,balcony,locality_name,airports_nearest,cityCenters_nearest,parks_around3000,ponds_around3000,days_exposition,area_group
0,20,13000000.0,108.0,2019-03-07,3,2.7,16.0,51.0,8,False,False,25.0,0,Санкт-Петербург,18863.0,16028.0,1,2,NaN,100 - 110


# **6. Обработка названий**

В имеющемся датасете колонка 'locality_name' имеет повторяющиеся названия, а также у некоторых значений перепутан тип поселения. Чтоб в дальнейшем была возможность пользоваться этим столбцом удалим из списка все указания на тип поселений, и заменим часть очевидных опечаток.

Для этого создадим словарь для удаления и замены слов:

In [ ]:
dict_for_delete = {'поселок ':'','посёлок ':'','деревня ':'','городского типа ':'',
                   'городской ':'','коттеджный ':'',"село ":"",
                   "садоводческое некоммерческое товарищество ":"",
                   'садовое товарищество ':'', "при железнодорожной ": "",
                   "станции":"станция", 'Саперный':"Саперное"}

Для простоты напишем функцию замены нескольких значений из словаря:

In [ ]:
# Функция для замены нескольких значений
def multiple_replace(target_str, replace_values):
    # получаем заменяемое: подставляемое из словаря в цикле
    for i, j in replace_values.items():
        # меняем все target_str на подставляемое
        target_str = target_str.replace(i, j)
    return target_str


In [ ]:
data['locality_name'] = [multiple_replace(element_in_locality_name, dict_for_delete)
                         if type(element_in_locality_name) == str 
                         else element_in_locality_name
                         for element_in_locality_name in data['locality_name']]

Выведем полученные уникальные названия мест:

In [ ]:
data.locality_name.unique()

array(['Санкт-Петербург', 'Шушары', 'Янино-1', 'Парголово', 'Мурино',
       'Ломоносов', 'Сертолово', 'Петергоф', 'Пушкин', 'Кудрово',
       'Коммунар', 'Колпино', 'Красный Бор', 'Гатчина', 'Фёдоровское',
       'Выборг', 'Кронштадт', 'Кировск', 'Новое Девяткино',
       'Металлострой', 'Лебяжье', 'Сиверский', 'Молодцово',
       'Кузьмоловский', 'Новая Ропша', 'Павловск', 'Пикколово',
       'Всеволожск', 'Волхов', 'Кингисепп', 'Приозерск', 'Сестрорецк',
       'Куттузи', 'Аннино', 'Ефимовский', 'Плодовое', 'Заклинье',
       'Торковичи', 'Первомайское', 'Красное Село', 'Понтонный',
       'Сясьстрой', 'Старая', 'Лесколово', 'Новый Свет', 'Сланцы',
       'Путилово', 'Ивангород', 'Шлиссельбург', 'Никольское',
       'Зеленогорск', 'Сосновый Бор', 'Оржицы', 'Кальтино', 'Романовка',
       'Бугры', 'Рощино', 'Кириши', 'Луга', 'Волосово', 'Отрадное',
       'Павлово', 'Оредеж', 'Копорье', 'Молодёжное', 'Тихвин', 'Победа',
       'Нурма', 'Синявино', 'Тосно', 'Стрельна', 'Бокситогорск',

# **7. Добавление необходимой информации в таблицу**

Представленный датасет не несет некоторых необходимых данных и требуется конструирование некоторых признаков:

* цену квадратного метра (столбец 'cost_per_square');
* день недели, месяц и год публикации объявления (столбец 'week', 'month', 'year');
* этаж квартиры; варианты — первый, последний, другой (столбец 'type_floor');
* соотношение жилой и общей площади, а также отношение
площади кухни к общей (столбец 'living_to_total', 'kitchen_to_total).


7.1 добавление стоимость квадратного метра жилья

In [ ]:
data['cost_per_square'] = data['last_price'] / data['total_area']

7.2 добавление дня недели, месяца и года

In [ ]:
data['week'] = data['first_day_exposition'].dt.day_name()
data['month'] = data['first_day_exposition'].dt.month
data['year'] = data['first_day_exposition'].dt.year

7.3 добавление типа этажа квартиры; варианты — первый, последний, другой (столбец 'type_floor');

In [ ]:
data['type_floor'] = ['last' if data.loc[row,'floors_total'] == data.loc[row,'floor']
                      else 'first' if data.loc[row,'floor'] == 1
                      else 'middle' for row in range(len(data))]

7.4 добавление соотношения жилой и общей площади и отношения
площади кухни к общей (столбец 'living_to_total', 'kitchen_to_total).

In [ ]:
data['living_to_total'] = data['living_area'] / data['total_area']
data['kitchen_to_total'] = data['kitchen_area'] / data['total_area']

# **8. Анализ времени продаж**

Проанализируем типичное время продажи квартиры, для этого построим гистограмму и boxplot распределения количества дней.

In [ ]:
fig = px.histogram(data, x='days_exposition')
fig.show() 


In [ ]:
fig = px.box(data, y="days_exposition")
fig.show()

Из полученных распределений видно, что характерное время продажи квартиры это от 45 до 231 дня. Значит продажи дольше 231 дня можно считать медленными, а продажи меньше чем, за 45 дней очень быстрыми. 


# **9. Выделение поселков с наибольшим числом объявлений, а также центральных квартир в СПб.**

**9.1 Выделим 10 населенных пунктов с наибольшим числом объявлений.**

Выберем 10 населённых пунктов с наибольшим числом объявлений. Для этого построим сводную таблицу для всех поселков и сохраним первые десять в отдельный список.

In [ ]:
table1 = pd.pivot_table(data, values='total_area', index='locality_name', aggfunc='count')

In [ ]:
sorted_table1 = table1.sort_values(by='total_area', ascending=False)

In [ ]:
sorted_table1

,total_area
locality_name,
Санкт-Петербург,15503
Мурино,584
Кудрово,469
Шушары,439
Всеволожск,397
...,...
Курковицы,1
Пчева,1
Старое Хинколово,1


In [ ]:
short_table = sorted_table1.iloc[:10, :]

In [ ]:
list_place = list(short_table.index)

Таким образом, мы получили список из топ 10 населенных пунктов с наибольшим числов объявлений.

In [ ]:
list_place

['Санкт-Петербург',
 'Мурино',
 'Кудрово',
 'Шушары',
 'Всеволожск',
 'Пушкин',
 'Колпино',
 'Парголово',
 'Гатчина',
 'Выборг']

Посмотрим на среднюю стоимость жилья в этих населенных пунктах:

In [ ]:
temp_data = data[data['locality_name'].isin(list_place)]

In [ ]:
temp_data.pivot_table(values='cost_per_square', index='locality_name', 
                      aggfunc='mean').sort_values(by='cost_per_square', 
                                                  ascending=False)

,cost_per_square
locality_name,
Санкт-Петербург,113576.329133
Пушкин,103100.890905
Кудрово,95261.963450
Парголово,90332.263023
Мурино,86018.110911
Шушары,78551.344545
Колпино,75333.295801
Гатчина,68757.678644
Всеволожск,68719.320400


**9.2 Выделим квартиры в центре с помощью стоимости среднего квадратного метра общей площади.**

Для этого создадим колонку указывающую на расстояние от центра в км.

In [ ]:
data['cityCenters_nearest_km'] = data['cityCenters_nearest'] // 1000

In [ ]:
table2 = pd.pivot_table(data, values='cost_per_square', index='cityCenters_nearest_km', aggfunc='mean')

In [ ]:
table2 = table2.reset_index()

Построим зависимость средней стоимости квадратного метра от км до центра.

In [ ]:
fig = px.line(table2, x="cityCenters_nearest_km", y="cost_per_square")
fig.show()

Итого центром можно считать квартиры расположенные в радиусе 6 км. Первый пик, а потом спад.

Создадим датафрейм для центральных квартир и остальных.

In [ ]:
data_center = data[data['cityCenters_nearest_km'] <= 6]
data_other = data[data['cityCenters_nearest_km'] > 6]

# **10. Анализ площади, цены, числа комнат и высоты потолков для квартир в центре.**

# 10.1 Анализ распределения площади квартир.

Построим гистограмму распределения площади квартир в центре:

In [ ]:
fig1 = px.histogram(data_center, x="total_area")
fig1.show() 

Выделяется несколько пиков, посмотрим подробнее данные по типам квартир:

In [ ]:
pd.pivot_table(data_center, values='total_area', columns='rooms', 
               aggfunc='describe')

rooms,0,1,2,3,4,5,6
25%,23.45000,35.000000,52.000000,74.000000,94.600000,122.650000,153.100000
50%,33.20000,42.050000,63.000000,87.000000,111.800000,149.000000,180.000000
75%,50.90000,48.000000,73.950000,106.125000,135.000000,178.850000,219.000000
count,12.00000,478.000000,963.000000,1016.000000,461.000000,199.000000,75.000000
max,98.40000,371.000000,196.000000,363.100000,317.000000,342.000000,312.000000
mean,42.22500,44.433996,65.530800,94.508179,119.368243,157.117638,188.744000
min,15.50000,12.000000,20.000000,43.000000,60.000000,62.200000,96.000000
std,25.66551,21.463948,18.567272,31.757068,36.450527,48.139151,53.406051


**Выводы:**
1. Однокомнатные квартиры и студии имеют в среднем одинаковую площадь.
2. В среднем разница с увеличением числа комнат это 20-30 кв.м.


# 10.2 Анализ распределения цены.


Построим гистограмму распределения цен квартир в центре в зависимости от числа комнат:

In [ ]:
fig = px.histogram(data_center, x="last_price",color='rooms')
fig.show() 


Для текстового восприятия средних цен построим сводную таблицу.

In [ ]:
pd.pivot_table(data_center, values='last_price', columns='rooms', 
               aggfunc='describe')

rooms,0,1,2,3,4,5,6
25%,3.175000e+06,4.800000e+06,5.965000e+06,7.660000e+06,8.900000e+06,1.099500e+07,1.315000e+07
50%,5.424500e+06,6.100000e+06,7.800000e+06,9.997000e+06,1.199000e+07,1.420000e+07,1.900000e+07
75%,6.500000e+06,7.437776e+06,1.067000e+07,1.515000e+07,1.828000e+07,2.500000e+07,2.740000e+07
count,1.200000e+01,4.780000e+02,9.630000e+02,1.016000e+03,4.610000e+02,1.990000e+02,7.500000e+01
max,1.630000e+07,7.100000e+07,7.761208e+07,9.900000e+07,9.600000e+07,9.500000e+07,9.000000e+07
mean,5.862083e+06,6.984110e+06,9.650605e+06,1.413619e+07,1.680815e+07,2.092188e+07,2.454296e+07
min,2.176000e+06,1.686000e+06,2.800000e+06,1.600000e+06,2.100000e+06,1.800000e+06,4.390000e+06
std,3.806347e+06,5.248740e+06,6.889577e+06,1.205808e+07,1.420420e+07,1.641822e+07,1.748275e+07


Средние цены на квартиры с разным числом комнат растут более или менее равномерно. Говорить о скачке цен между пяти- и шестикомнатными квартирами как о точном значении нельзя, так как мало таких квартир для проверки гипотезы.



# 10.3 Анализ распределения числа комнат.


Построим гистограмму распределения квартир в центре в зависимости от числа комнат.

In [ ]:
fig = px.histogram(data_center, x="rooms")
fig.show() 


In [ ]:
fig = px.histogram(data_other, x="rooms")
fig.show() 

В центре мы в основном сталкиваемся с двух или трехкомнатными квартирами, в то время как в остальной части Петербурга и области преобладают однокомнатные квартиры.

# 10.4 Анализ распределения высоты потолков.


Построим гистограмму распределения высоты полотков:

In [ ]:
fig = px.histogram(data_center, x="ceiling_height")
fig.show() 


Выделяется резкий пик для квартир с высотой потолка в 2.7 м. Таких квартир большиснтво. Также стоит отмеить, что график в большинстве своем дискретный. Это говорит о том, что высота потолка формируется еще на стадии проекта дома и одинакова для этого типа построек.

# **11. Какие факторы влияют на формирование цены в центре?**

Выделим основные параметры, которые могут влиять на цену квартиры и построим их корреляцию:

In [ ]:
temp_list_columns = [ 'last_price', 'total_area', 'rooms', 'ceiling_height', 
                'floors_total', 'floor', 'studio', 'open_plan', 'balcony',
                'cityCenters_nearest', 'parks_around3000', 'ponds_around3000', 
                'days_exposition', 'type_floor']

In [ ]:
corr = data_center[temp_list_columns].corr()
corr.style.background_gradient(cmap='coolwarm').set_precision(2)

,last_price,total_area,rooms,ceiling_height,floors_total,floor,studio,open_plan,balcony,cityCenters_nearest,parks_around3000,ponds_around3000,days_exposition
last_price,1.00,0.71,0.37,0.15,0.10,0.15,-0.04,0.01,0.10,-0.03,0.14,0.09,0.11
total_area,0.71,1.00,0.74,0.23,-0.00,0.10,-0.06,0.01,0.02,-0.09,0.10,-0.00,0.16
rooms,0.37,0.74,1.00,0.21,-0.18,-0.04,-0.14,-0.03,-0.08,-0.17,0.10,-0.04,0.14
ceiling_height,0.15,0.23,0.21,1.00,-0.26,-0.15,0.04,0.01,-0.08,-0.20,0.14,0.01,0.08
floors_total,0.10,-0.00,-0.18,-0.26,1.00,0.65,-0.01,0.00,0.37,0.34,-0.25,-0.04,0.02
floor,0.15,0.10,-0.04,-0.15,0.65,1.00,-0.02,-0.01,0.29,0.21,-0.16,-0.05,0.03
studio,-0.04,-0.06,-0.14,0.04,-0.01,-0.02,1.00,-0.00,0.02,0.01,-0.04,0.01,-0.04
open_plan,0.01,0.01,-0.03,0.01,0.00,-0.01,-0.00,1.00,-0.02,0.01,0.01,-0.01,-0.03
balcony,0.10,0.02,-0.08,-0.08,0.37,0.29,0.02,-0.02,1.00,0.17,-0.12,0.04,0.00
cityCenters_nearest,-0.03,-0.09,-0.17,-0.20,0.34,0.21,0.01,0.01,0.17,1.00,-0.21,-0.33,-0.04


Основные факторы влияющие на ценообразование, это общая площадь и количество комнат. Стоит отметить, что в таблицу не вошел критерий разделения квартир по этажам. Построим его отдельно. 

In [ ]:
pd.pivot_table(data_center, values=['last_price','cost_per_square'], index='type_floor', aggfunc='mean')

,cost_per_square,last_price
type_floor,,
first,120259.965045,8.505266e+06
last,133071.974582,1.308177e+07
middle,144871.941991,1.333489e+07


**Выводы**

1. Ценообразование в большей степени зависит: от общей площади, от количества комнат.
2. Есть слабая корреляция (0.15) от того на каком этаже находится квартира,высота потолков, количество парков рядом или этаж квартиры.
3. Такие критерии как количество прудов рядом или балконов почти не влияют на цену квартиры.
4. Ценообразование квартир зависит от этажа на котором они располагаются, первый этаж это самые дешевые квартиры, затем идет последний этаж. И самые дорогие на промежуточных этажах.

# **12. Факторы ценообразования вне центра.**

Рассмотрим на факторы влияющие на ценообразование вне центра, будут ли они отличаться.

In [ ]:
corr = data_other[temp_list_columns].corr()
corr.style.background_gradient(cmap='coolwarm').set_precision(2)

,last_price,total_area,rooms,ceiling_height,floors_total,floor,studio,open_plan,balcony,cityCenters_nearest,parks_around3000,ponds_around3000,days_exposition
last_price,1.00,0.80,0.49,0.34,0.11,0.10,-0.05,-0.01,0.09,-0.19,0.12,0.16,0.10
total_area,0.80,1.00,0.76,0.30,0.04,0.04,-0.08,-0.04,0.09,-0.04,0.06,0.11,0.13
rooms,0.49,0.76,1.00,0.07,-0.18,-0.13,-0.16,-0.04,0.05,-0.02,0.04,0.03,0.08
ceiling_height,0.34,0.30,0.07,1.00,0.10,0.08,0.03,0.01,0.03,-0.06,0.11,0.13,0.07
floors_total,0.11,0.04,-0.18,0.10,1.00,0.65,0.07,0.05,0.14,-0.20,-0.19,-0.05,-0.01
floor,0.10,0.04,-0.13,0.08,0.65,1.00,0.03,0.04,0.15,-0.13,-0.12,-0.02,-0.02
studio,-0.05,-0.08,-0.16,0.03,0.07,0.03,1.00,-0.00,0.03,-0.02,-0.03,0.00,-0.02
open_plan,-0.01,-0.04,-0.04,0.01,0.05,0.04,-0.00,1.00,0.03,-0.01,-0.02,0.01,-0.02
balcony,0.09,0.09,0.05,0.03,0.14,0.15,0.03,0.03,1.00,-0.02,-0.03,-0.01,-0.02
cityCenters_nearest,-0.19,-0.04,-0.02,-0.06,-0.20,-0.13,-0.02,-0.01,-0.02,1.00,-0.02,0.12,0.02


In [ ]:
pd.pivot_table(data_other, values=['last_price','cost_per_square'], index='type_floor', aggfunc='mean')

,cost_per_square,last_price
type_floor,,
first,90643.545261,4.988396e+06
last,98756.370822,5.794084e+06
middle,105257.801244,5.979733e+06


**Выводы**

1. Ценообразование в большей степени зависит: от общей площади, от количества комнат. В отличие от центральных квартир добавляется еще и высота потолка.
2. Возникает также отрицательная корреляция с расстоянием до центра города (-0.19).
3. Есть слабая корреляция (0.15) от количества прудов рядом. В отличие от количества парков рядом, что свойственно для центра.
4. Ценообразование квартир зависит от этажа на котором они располагаются, первый этаж это самые дешевые квартиры, затем идет последний этаж. И самые дорогие на промежуточных этажах.
5. Наличие балкона, этаж квартиры и наличие парков рядов слабо влиют около 0.10